In [109]:
import os, sys, pandas as pd, datetime
module_path = os.path.abspath(os.path.join('/Users/ktereshin/Desktop/yandex/analytics/python/lib'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('/Users/ktereshin/Desktop/yandex/analytics/python/work'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from vault_client import instances
from global_variables import (
    metrika_clickhouse_param_dict,
    cloud_clickhouse_param_dict
)
from nile.api.v1 import (
    clusters,
    aggregators as na,
    extractors as ne,
    filters as nf,
    Record
)
from data_loader import clickhouse

In [110]:
client = instances.Production()
metrika_creds = client.get_version('ver-01d2z36msatt9mp9pcfptezksp')
yc_ch_creds = client.get_version('ver-01d2z39xj02xw7gqvv9wq757ne')
cloud_clickhouse_param_dict['user'] = yc_ch_creds['value']['login']
cloud_clickhouse_param_dict['password'] = yc_ch_creds['value']['pass']

In [111]:
yt_creds = client.get_version('ver-01d33pgv8pzc7t99s3egm24x47')
cluster = clusters.yt.Hahn(
    token = yt_creds['value']['token'],
    pool = yt_creds['value']['pool']
)

In [112]:
sample = cluster.read('//home/cloud_analytics/cubes/acquision_cube/2019-02-11')

In [ ]:
i = 0
query = ''
schema = 'cloud_analytics_testing'
table = 'test_cube'
for row in sample:
    if row['event_time']:
        i += 1
        print(i)
        row_dict = row.to_dict()
        if not columns:
            columns = sorted(list(row_dict.keys()))
            
        for key in columns:
            if row_dict[key]:
                if isinstance(row_dict[key], str):
                    values.append("'" + row_dict[key] + "'")
                else:
                    values.append(str(row_dict[key]))
            else:
                values.append('NULL')
    values_str = values_str + '(' + ','.join(values) + ')'
    if query:
        query = '''INSERT INTO {0}.{1} ({2}) VALUES {3}'''.format(schema,table,','.join(columns), values_str)
        
    else:
        query = query + ',' + values_str
        
    if i%10 == 0:
        cloud_clickhouse_param_dict['query'] = query
        clickhouse.clickhouse_request(**cloud_clickhouse_param_dict)
        query = ''
        i =+ 1
        break

1
2
3
4
5
6
7
8
9


In [94]:
columns = []
values = []
values_str = ''
columns_str = ''
row_dict = row.to_dict()
columns = sorted(list(row_dict.keys()))
for key in columns:
    if row_dict[key]:
        if isinstance(row_dict[key], str):
            values.append("'" + row_dict[key] + "'")
        else:
            values.append(str(row_dict[key]))
    else:
        values.append('NULL')
values_str = values_str + '(' + ','.join(values) + ')'

In [95]:
query = '''INSERT INTO {0}.{1} ({2}) VALUES {3}'''.format(schema,table,','.join(columns), values_str)

In [97]:
cloud_clickhouse_param_dict['user'] = yc_ch_creds['value']['login']
cloud_clickhouse_param_dict['password'] = yc_ch_creds['value']['pass']
cloud_clickhouse_param_dict['query'] = query
clickhouse.clickhouse_request(**cloud_clickhouse_param_dict)

<Response [200]>

In [ ]:
cloud_clickhouse_param_dict['user'] = cloud_clickhouse_param_dict['value']['login']
cloud_clickhouse_param_dict['password'] = cloud_clickhouse_param_dict['value']['pass']
cloud_clickhouse_param_dict['query'] = query